# Import Library

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load Dataset

In [8]:
# Load dataset
df = r"D:\Analyst_Portofolio\Most_Streamed_Spotify_Song\data_raw\spotify-2023.csv"

In [9]:
df

'D:\\Analyst_Portofolio\\Most_Streamed_Spotify_Song\\data_raw\\spotify-2023.csv'

# Pre-Processing

In [10]:
# Load the CSV file into a DataFrame if not already loaded
if isinstance(df, str):
	df = pd.read_csv(df, encoding='latin1')

# Tambahkan ID unik untuk setiap lagu
df['track_id'] = range(len(df))

In [11]:
# Simulasi 50 pengguna, masing-masing menyukai 10-30 lagu secara acak
num_users = 50
np.random.seed(42)
user_ids = [f'user_{i+1}' for i in range(num_users)]
user_track_matrix = pd.DataFrame(0, index=user_ids, columns=df['track_id'])

for user in user_track_matrix.index:
    liked_ids = np.random.choice(df['track_id'], size=np.random.randint(10, 30), replace=False)
    user_track_matrix.loc[user, liked_ids] = 1

# Hitung Similarity

In [12]:
# Hitung cosine similarity antar pengguna
user_similarity = cosine_similarity(user_track_matrix.values)
user_similarity_df = pd.DataFrame(user_similarity, index=user_track_matrix.index, columns=user_track_matrix.index)

def recommend_collaborative(user_id, top_n=5):
    if user_id not in user_track_matrix.index:
        return f"User {user_id} tidak ditemukan."

    sim_scores = user_similarity_df[user_id]
    other_users_matrix = user_track_matrix.drop(index=user_id)
    sim_scores_others = sim_scores.drop(index=user_id)

    scores = other_users_matrix.T.dot(sim_scores_others)
    scores = scores / sim_scores_others.sum()

    user_interactions = user_track_matrix.loc[user_id]
    unseen_scores = scores[user_interactions == 0]

    top_recs = unseen_scores.sort_values(ascending=False).head(top_n).index
    return df[df['track_id'].isin(top_recs)][['track_name', 'artist(s)_name']]

# Pengujian

In [13]:
# Contoh penggunaan
if __name__ == "__main__":
    print("Sistem Rekomendasi Lagu - Collaborative Filtering")
    user_input = input("Masukkan user ID (contoh: user_1): ")
    results = recommend_collaborative(user_input)
    print("\nRekomendasi:")
    print(results)

Sistem Rekomendasi Lagu - Collaborative Filtering

Rekomendasi:
User 2 tidak ditemukan.
